# Model Editing

We use our IG and AP pipeline to localise important components. These components are edited using gradient descent to "unlearn" information. We evaluate our results on the CounterFact dataset.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor
import torch.optim as optim

from testing import logit_diff_metric
from applications.pipeline import run_attribution_steps, identify_target_components, optimise_edit_components, AttributionMethod, edit_model
from applications.datasets import CounterFact

from transformer_lens import HookedTransformer
from transformer_lens.utils import get_device
import copy

/vol/bitbucket/mwl21/fypvenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = get_device()
# device = torch.device("cpu")
model = HookedTransformer.from_pretrained("gpt2-small", device=device)

# Explicitly calculate and expose the result for each attention head
model.set_use_attn_result(True)
model.set_use_hook_mlp_in(True)

Loaded pretrained model gpt2-small into HookedTransformer


## Editing procedure

In [4]:
# Verify that loading works, for one example
n_samples = 5

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))
# clean_input, corrupted_input, labels = counterfact_dataset.get_single_sample(0)

# print(clean_input)
# print(corrupted_input)
# print(labels)


collate tensor([[[24111],
         [15823]]], device='cuda:0')


In [26]:
from applications.pipeline import localise_models

counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=n_samples)

clean_input, corrupted_input, labels = next(iter(counterfact_dataloader))

target_mlp, target_attn = localise_models(model, clean_input, corrupted_input, labels, overwrite=False)

collate tensor([[[24111],
         [15823]]], device='cuda:0')
torch.Size([10, 12, 3072])


In [42]:
from applications.metrics import evaluate_counterfact_efficacy, evaluate_counterfact_paraphrased, evaluate_counterfact_neighborhood, evaluate_consistency
from applications.datasets import CounterFact
import pandas as pd
from collections import defaultdict

evaluation_scores = defaultdict(list)
counterfact_dataset = CounterFact(model)
counterfact_dataloader = counterfact_dataset.to_dataloader(batch_size=1)

for n, (clean_input, corrupted_input, labels) in enumerate(counterfact_dataloader):

    original_output = model.generate(clean_input, max_new_tokens=5, do_sample=False)

    edited_model = edit_model(model, clean_input, corrupted_input, labels, target_mlp[n], target_attn[n], n_epochs=10)

    # Evaluate
    print(f"Prompt: {clean_input}")
    print("Original output:", original_output)
    # print(f"Original answer: {labels[:, 0]}. Target answer: {labels[:, 1]}")

    print(edited_model.generate(clean_input, max_new_tokens=5, do_sample=False))

    score, magnitude = evaluate_counterfact_efficacy(edited_model, n, verbose=True)
    evaluation_scores["Efficacy score"].append(score.item())
    evaluation_scores["Efficacy magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_paraphrased(edited_model, n, verbose=False)
    evaluation_scores["Generalisation score"].append(score.item())
    evaluation_scores["Generalisation magnitude"].append(magnitude.item())

    score, magnitude = evaluate_counterfact_neighborhood(edited_model, n, verbose=False)
    evaluation_scores["Specificity score"].append(score.item())
    evaluation_scores["Specificity magnitude"].append(magnitude.item())

    consistency_score = evaluate_consistency(model, n, verbose=False)
    evaluation_scores["Consistency score"].append(score.item())
    evaluation_scores["Consistency magnitude"].append(magnitude.item())

    if n + 1 >= n_samples: break

evaluation_df = pd.DataFrame(evaluation_scores)
evaluation_df.to_csv('results/counterfact/evaluation.csv')

collate tensor([[[24111],
         [15823]]], device='cuda:0')


100%|██████████| 5/5 [00:00<00:00, 55.89it/s]


Fine tuning model...
Target MLP tensor(656, device='cuda:0')
Target attn tensor(17, device='cuda:0')


Total loss: 13.766560554504395, forget loss: 0.8218996524810791, retain loss: 12.944661140441895
Total loss: 9.437654495239258, forget loss: 0.6728930473327637, retain loss: 8.764761924743652
Total loss: 6.063858509063721, forget loss: 0.3877338171005249, retain loss: 5.676124572753906
Total loss: 1.5688538551330566, forget loss: 0.38302385807037354, retain loss: 1.185829997062683
Total loss: 0.7483470439910889, forget loss: 0.7088928818702698, retain loss: 0.0394541397690773
Total loss: 0.04736701399087906, forget loss: 0.04024618864059448, retain loss: 0.007120825815945864
Total loss: 0.0001881093776319176, forget loss: 0.00011229515075683594, retain loss: 7.581423415103927e-05
Total loss: 5.2452069212449715e-06, forget loss: 3.2186508178710938e-06, retain loss: 2.0265558760002023e-06
Total loss: 2.384185791015625e-07, forget loss: 1.1920928955078125e-07, retain loss: 1.1920928244535389e-07
Total loss: 0.0, forget loss: 0.0, retain loss: 0.0
Prompt: ['The mother tongue of Danielle Da

100%|██████████| 5/5 [00:00<00:00, 58.11it/s]

...


Fine tuning model...
Target MLP tensor(946, device='cuda:0')
Target attn tensor(6, device='cuda:0')
Total loss: 20.040800094604492, forget loss: 0.8302270174026489, retain loss: 19.210573196411133
Total loss: 15.194461822509766, forget loss: 0.877143383026123, retain loss: 14.317317962646484
Total loss: 11.515726089477539, forget loss: 0.910816490650177, retain loss: 10.604909896850586
Total loss: 7.953537940979004, forget loss: 0.9416232109069824, retain loss: 7.0119147300720215
Total loss: 4.3495635986328125, forget loss: 0.971041202545166, retain loss: 3.3785221576690674
Total loss: 1.6103942394256592, forget loss: 0.799092710018158, retain loss: 0.8113014698028564
Total loss: 0.19345290958881378, forget loss: 0.15358585119247437, retain loss: 0.039867062121629715
Total loss: 0.005476599559187889, forget loss: 0.004637718200683594, retain loss: 0.0008388814167119563
Total loss: 0.00018560848548077047, forget loss: 0.00015807151794433594, retain loss: 2.753696753643453e-05
Total los

100%|██████████| 5/5 [00:00<00:00, 58.81it/s]

What is the twin city of Lyon? It isManilaManilaMan


Original label: Beirut
Target label: Manila
Outputs: [' its', ' its', ' the', ' the', ' the', ' the', ' its', ' the', ' the', ' the']


100%|██████████| 5/5 [00:00<00:00, 48.45it/s]


## Evaluation

For each sample, we calculate the efficacy, generalisability, specificity and consistency for:

- The original models' outputs
- The edited model's outputs


In [41]:
evaluation_df.head()

,Efficacy score,Efficacy magnitude,Generalisation score,Generalisation magnitude,Specificity score,Specificity magnitude,Consistency score,Consistency magnitude
0,0.0,-0.000024,0.0,-0.000023,0.0,-0.000036,0.0,-0.000036
1,0.0,-0.000007,0.0,-0.000004,0.0,-0.000003,0.0,-0.000003
2,0.6,0.000012,0.5,0.000010,0.9,0.000015,0.9,0.000015
3,0.7,0.000041,0.5,0.000035,0.9,0.000045,0.9,0.000045
4,0.0,-0.000022,0.0,-0.000020,0.1,-0.000119,0.1,-0.000119
